In [ ]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [ ]:
resultados = pd.read_csv("csv_originales/results.csv", engine="python") # Read CSV file with specified encoding and separator

## ETL results

In [ ]:
resultados

In [ ]:
resultados.info()

In [ ]:
resultados.isnull().sum()

In [ ]:
# Creo un codigo para que me borre todas las filas que NO contengan el nombre de Boca Juniors en el dataset de resultados
resultados = resultados[resultados['local_team'].str.contains("Boca Juniors") | resultados['visitor_team'].str.contains("Boca Juniors")]

In [ ]:
# Elimino la columna llamada "Unnamed: 0" que no es necesaria
resultados = resultados.drop(columns=["Unnamed: 0"])

In [ ]:
# Extraigo el año de la columna 'date_name' y lo guardo en una nueva columna llamada 'year'
resultados['year'] = resultados['date_name'].str.extract(r'(\b(?:19|20)\d{2}\b)')

In [ ]:
# Guardo el dataframe en un nuevo CSV dentro de la carpeta "csv_limpios"
resultados.to_csv("csv_limpios/resultados_boca_juniors.csv", index=False, encoding="utf-8-sig")

-------------------

In [ ]:
res_l = pd.read_csv("csv_limpios/resultados_boca_juniors.csv") # Read the cleaned CSV file

In [ ]:
res_l

## ETL datos_partidos_boca_juniors

In [ ]:
datos_bj = pd.read_csv("csv_originales/datos_partidos_boca_juniors.csv", engine="python") # Read CSV file with specified encoding and separator

In [ ]:
datos_bj

In [ ]:
datos_bj.isnull().sum()

Dejo solo los datos de Boca Juniors

In [ ]:
# Creo un codigo para que me borre todas las filas que NO contengan el nombre de Boca Juniors en el dataset de datos_bj
datos_bj = datos_bj[datos_bj['1'].str.contains("Boca Juniors") | datos_bj['3'].str.contains("Boca Juniors")]

Reemplazo 'Argentine Copa de la Liga Profesional', 'Copa de Liga Profesional' en la columna '5'

In [ ]:
# Creo una funcion para reemplazar valores de filas
def replace_values(df, column, old_value, new_value):
    df[column] = df[column].replace(old_value, new_value)
    return df

In [ ]:
replace_values(datos_bj, '5', 'Argentine Copa de la Liga Profesional', 'Copa de Liga Profesional')

Columna NOTAS

In [ ]:
# Creo un codigo para crear una nueva columna llamada 'Notas' que contenga el texto de la columna '2' despues de los primeros 4 digitos
datos_bj['Notas'] = datos_bj['2'].str[5:]

Quedarme con solo los resultados en la columna '2'

In [ ]:
# Creo un codigo para quedarme solo con los primeros 3 digitos de la columna '2' 
datos_bj['2'] = datos_bj['2'].str[:5]

### Creo una columna de Fecha desglozando la columna '0'

Extraigo el mes de la columna '0' y creo una nueva llamada 'Mes'

In [ ]:
# Creo un codigo para extraer el mes de la columna en texto '0', el cual son los ultimos 3 digitos y guardarlo en una nueva columna llamada 'Mes'
datos_bj['Mes'] = datos_bj['0'].str[-4:-1]

In [ ]:
# Cambio el mes de la columna 'Mes' a su numero correspondiente sin importar mayusculas o minusculas
meses = {
    'ene': '01', 'feb': '02', 'mar': '03', 'abr': '04',
    'may': '05', 'jun': '06', 'jul': '07', 'ago': '08',
    'sep': '09', 'oct': '10', 'nov': '11', 'dic': '12'
}
datos_bj['Mes'] = datos_bj['Mes'].str.lower().map(meses)

Extraigo el dia de la columna '0' y creo una nueva llamada 'Dia'

In [ ]:
# Extraigo el dia de la columna '0' y lo guardo en una nueva columna llamada 'Dia'
datos_bj['Dia'] = datos_bj['0'].str[-10:-8]

In [ ]:
# Con las columnas 'Dia' y 'Mes' creo una nueva columna llamada 'Fecha' con el formato 'DD/MM/2023' 
datos_bj['Fecha'] = datos_bj['Dia'] + '/' + datos_bj['Mes'] + '/2023'
# Paso a formato fecha la columna 'Fecha'
datos_bj['Fecha'] = pd.to_datetime(datos_bj['Fecha'], format='%d/%m/%Y')

Borro columna '0', 'Dia' y 'Mes'

In [ ]:
datos_bj.drop(columns=['0', 'Dia', 'Mes'], inplace=True)

Reemplazo nombres de columnas para que sea mas legible.

In [ ]:
datos_bj

In [ ]:
# Reemplazo nombres de columnas para que sea mas legible.
datos_bj.columns = ['Local', 'Resultado', 'Visitante', 'Status', 'Competition', 'Notas', 'Fecha']

Cargo csv a carpeta CSV_LIMPIOS

In [ ]:
datos_bj.to_csv("csv_limpios/datos_bj23.csv")

--------------------------------

### Ultimos cambios en los csv de la carpeta csv_limpios (datos_bj23 y resultados_boca_juniors)

In [ ]:
datos_23 = pd.read_csv("csv_limpios/datos_bj23.csv") # Read the cleaned CSV file
res_l = pd.read_csv("csv_limpios/resultados_boca_juniors.csv") # Read the cleaned CSV file

## datos_23 (datos_bj23)

In [ ]:
datos_23

In [ ]:
datos_23.drop(columns=['Unnamed: 0'], inplace=True) # Drop unnecessary column

In [ ]:
# Con el dataframe guardado en la variable equipos con las id de los equipos, creo una nueva columna en datos_23 llamada 'local_team_id' que contenga el id del equipo local que coincida con el nombre del equipo local en la columna 'Local', y lo mismo con 'Visitante' 
equipos_local2 = equipos[equipos['team_name'].isin(datos_23['Local'])][['team_id', 'team_name']]
equipos_visitante2 = equipos[equipos['team_name'].isin(datos_23['Visitante'])][['team_id', 'team_name']]
# Con las variables creadas creo 2 columnas nuevas en datos_23 llamadas 'local_team_id' y 'visitor_team_id' que contengan el id del equipo local y visitante respectivamente
datos_23['local_team_id'] = datos_23['Local'].map(equipos_local2.set_index('team_name')['team_id'])
datos_23['visitor_team_id'] = datos_23['Visitante'].map(equipos_visitante2.set_index('team_name')['team_id'])


In [ ]:
equipos_local2

## res_l (resultados_boca_juniors)

In [ ]:
res_l

In [ ]:
# Con las columnas de local_team_id, local_team y visitor_team y visitor_team_id creo un array donde me muestre el id del equipo y el nombre del equipo
equipo_local = resultados[['local_team_id', 'local_team']].drop_duplicates().rename(columns={'local_team_id': 'team_id', 'local_team': 'team_name'})
equipo_visitante = resultados[['visitor_team_id', 'visitor_team']].drop_duplicates().rename(columns={'visitor_team_id': 'team_id', 'visitor_team': 'team_name'})
equipos = pd.concat([equipo_local, equipo_visitante]).drop_duplicates().reset_index(drop=True)
# Con el array de equipos creo un nuevo dataframe donde me muestre el id del equipo y el nombre del equipo
equipos_df = pd.DataFrame(equipos)


In [ ]:
equipos

Creo columna Fecha/Grupo extrayendo de date_name los ultimos 8 digitos

In [ ]:
res_l['Fecha/Grupo'] = res_l['date_name'].str[-8:]

In [ ]:
res_l['Evento'] = res_l['date_name'].str[:-10]

Cargo el csv modificado de la variable res_l al csv resultados_boca_juniors en la carpeta csv_limpios

In [ ]:
res_l.to_csv("csv_limpios/resultados_boca_juniors.csv", index=False, encoding="utf-8-sig") # Save the modified DataFrame to CSV